<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MISTRAL_OPENMODELDEMO_AAI_AOCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Agentic AI refers to AI systems designed to operate autonomously, making decisions and taking actions to achieve specific goals with limited or no human supervision. Unlike traditional AI, which often operates within predefined rules and requires human input, agentic AI exhibits autonomy, goal-driven behavior, and adaptability. These systems utilize AI agents, which are machine learning models that mimic human decision-making to solve problems in real-time.

In [ ]:
!pip install mistralai -q
!pip install colab-env -q
import colab_env

In [2]:
import os
from mistralai import Mistral

In [3]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

api_key = os.environ.get("MISTRAL_API_KEY")

if not api_key:
    print("Error: MISTRAL_API_KEY environment variable not set.")
    print("Please set your Mistral API key before running this script.")
    exit()


model_list = client.models.list()

# Print all model IDs
#for model_info in model_list.data:
#    print(model_info.id)

model = "open-mixtral-8x22b"

messages = [{"role": "user", "content": "What is the best Cuban poet?"}]

chat_response = client.chat.complete(
    model=model,  # Pass the model ID string
    messages=messages,
)
print('\n')
print(chat_response.choices[0].message.content)


print('\n\n')

historical_context_agent = client.beta.agents.create(
    model = "open-mixtral-8x22b",
    description="Agent for providing historical context and the intellectual lineage of scientific discoveries, including the contributions of figures like Newton, Galileo, Einstein, and Hinton.",
    name="historical-context-agent",
    tools=[
        # Removed {"type": "web_search"} as magistral-medium-latest does not support built-in connectors
        {
            "type": "function",
            "function": {
                "name": "get_scientist_contributions",
                "description": "Retrieve key scientific contributions of a historical figure.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "scientist_name": {"type": "string", "description": "Name of the scientist (e.g., 'Isaac Newton', 'Albert Einstein')."},
                        "field_of_study": {"type": "string", "description": "Optional: Specific field of study to narrow down contributions."}
                    },
                    "required": ["scientist_name"]
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "analyze_intellectual_synergy",
                "description": "Analyze the intellectual synergy or influence between different historical scientific figures.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "figure_A": {"type": "string", "description": "First scientific figure."},
                        "figure_B": {"type": "string", "description": "Second scientific figure."},
                        "common_theme": {"type": "string", "description": "Optional: Common theme or area of influence (e.g., 'relativity', 'mechanics', 'AI')."}
                    },
                    "required": ["figure_A", "figure_B"]
                }
            }
        }
    ]
)
print(f"Historical Context Agent '{historical_context_agent.name}' created with ID: {historical_context_agent.id}")



Determining the "best" poet from any country can be subjective as it depends on personal preferences, but one of the most renowned Cuban poets is José Martí. He is not only celebrated for his poetry but also as a national hero and key figure in Cuba's struggle for independence. His poems are widely read and admired for their patriotic themes and lyrical beauty. Other notable Cuban poets include Nicolás Guillén, Dulce María Loynaz, and Nancy Morejón.



Historical Context Agent 'historical-context-agent' created with ID: ag_0197d861b76e74a288d2ce50d2b85716


## AGENTINC AI

In [4]:
# Imports remain the same
import os
import math
import random
from datetime import datetime, timedelta
from mistralai import Mistral

# --- Configuration ---
# Assuming API key setup from previous code
# MISTRAL_API_KEY and client initialization

MISTRAL_MODEL = "open-mixtral-8x22b"

# Simulated Knowledge Bases (remain the same, potentially loaded from a DB in real AOCC)
AIRCRAFT_DATA = {
    'BOEING 737': {'speed': 450, 'fuel_consumption_rate': 1500, 'range': 3000, 'optimal_altitude': 35000},
    'AIRBUS A320': {'speed': 470, 'fuel_consumption_rate': 1600, 'range': 3200, 'optimal_altitude': 37000},
    'CESSNA 172': {'speed': 120, 'fuel_consumption_rate': 8, 'range': 600, 'optimal_altitude': 8000},
    'BOEING 777': {'speed': 550, 'fuel_consumption_rate': 3000, 'range': 8000, 'optimal_altitude': 40000}
}

AIRPORT_DATA = {
    'JFK': {'name': 'John F. Kennedy International Airport', 'lat': 40.6413, 'lon': -73.7781, 'runways': 'Long, Multiple'},
    'LAX': {'name': 'Los Angeles International Airport', 'lat': 33.9416, 'lon': -118.4085, 'runways': 'Long, Multiple'},
    'ORD': {'name': 'O\'Hare International Airport', 'lat': 41.9742, 'lon': -87.9073, 'runways': 'Long, Multiple'},
    'MIA': {'name': 'Miami International Airport', 'lat': 25.7959, 'lon': -80.2870, 'runways': 'Long, Multiple'},
    'YYZ': {'name': 'Toronto Pearson International Airport', 'lat': 43.6777, 'lon': -79.6248, 'runways': 'Long, Multiple'},
    'YUL': {'name': 'Montréal-Trudeau International Airport', 'lat': 45.4706, 'lon': -73.7408, 'runways': 'Long, Multiple'},
    'LHR': {'name': 'London Heathrow Airport', 'lat': 51.4700, 'lon': -0.4543, 'runways': 'Long, Multiple'},
    'CDG': {'name': 'Paris Charles de Gaulle Airport', 'lat': 49.0097, 'lon': 2.5479, 'runways': 'Long, Multiple'},
    'ATL': {'name': 'Hartsfield-Jackson Atlanta International Airport', 'lat': 33.6407, 'lon': -84.4277, 'runways': 'Long, Multiple'},
    'DFW': {'name': 'Dallas/Fort Worth International Airport', 'lat': 32.8998, 'lon': -97.0403, 'runways': 'Long, Multiple'},
    'EWR': {'name': 'Newark Liberty International Airport', 'lat': 40.6895, 'lon': -74.1745, 'runways': 'Long, Multiple'},
    'BOS': {'name': 'Boston Logan International Airport', 'lat': 42.3643, 'lon': -71.0052, 'runways': 'Long, Multiple'},
    'PVG': {'name': 'Shanghai Pudong International Airport', 'lat': 31.1434, 'lon': 121.8053, 'runways': 'Long, Multiple'}
}

# --- Tool Service Class ---
class ToolService:
    def __init__(self):
        pass

    def calculate_distance(self, lat1, lon1, lat2, lon2):
        """Calculates the great-circle distance between two geographical points (airports) in miles."""
        R = 3958.8 # Radius of Earth in miles
        dLat = math.radians(lat2 - lat1)
        dLon = math.radians(lon2 - lon1)
        a = (math.sin(dLat / 2) * math.sin(dLat / 2) +
             math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
             math.sin(dLon / 2) * math.sin(dLon / 2))
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        distance = R * c
        return distance

    def get_simulated_weather(self, airport_code, severity='normal'):
        """Simulates weather conditions for a given airport."""
        base_weather = {
            'JFK': 'Clear skies, 15°C, winds 10 knots from NW',
            'LAX': 'Sunny, 22°C, winds 5 knots from SW',
            'ORD': 'Partly cloudy, 10°C, winds 12 knots from N',
            'MIA': 'Humid, 28°C, winds 8 knots from SE',
            'YYZ': 'Cloudy, 8°C, winds 15 knots from E',
            'YUL': 'Overcast, 5°C, winds 18 knots from NE',
            'LHR': 'Foggy, 7°C, winds 7 knots from W',
            'CDG': 'Rainy, 12°C, winds 20 knots from SW',
            'ATL': 'Scattered clouds, 20°C, winds 7 knots from S',
            'DFW': 'Sunny, 25°C, winds 10 knots from SE',
            'EWR': 'Partly cloudy, 18°C, winds 10 knots from W',
            'BOS': 'Clear skies, 12°C, winds 8 knots from SW',
            'PVG': 'Partly cloudy, 20°C, winds 10 knots from SE', # Simulated weather for Shanghai
        }
        weather = base_weather.get(airport_code.upper(), 'Unknown weather conditions.')
        if severity == 'bad':
            weather += ' **SEVERE THUNDERSTORMS EXPECTED! Visibility low.**'
        elif severity == 'moderate':
            weather += ' Moderate turbulence reported. Light rain.'
        return weather

# --- AOCC-Oriented Agent Roles (UPDATED) ---
AOCC_AGENT_ROLES = {
    'flight_request_processor': "You are an AOCC flight request intake specialist. Your role is to gather and clarify initial flight requirements from internal airline stakeholders.",
    'aircraft_resources_agent': "You are an AOCC aircraft resource manager. Provide detailed performance and availability data for specific aircraft types from the airline's fleet.",
    'airport_ops_agent': "You are an AOCC airport operations specialist. Provide comprehensive details about airports, including their operational status, available infrastructure (runways, gates), and local restrictions.",
    'flight_dispatch_agent': "You are an AOCC flight dispatcher. Your expertise is in calculating optimal flight routes, distances, and estimated flight times, considering airspace and operational constraints.",
    'origin_weather_desk': "You are an AOCC meteorologist specializing in departure airport conditions. Analyze current and forecasted weather for takeoff and initial climb phases.",
    'destination_weather_desk': "You are an AOCC meteorologist specializing in destination airport conditions. Analyze current and forecasted weather for approach, landing, and potential hazards.",
    'enroute_weather_desk': "You are an AOCC en-route meteorologist. Analyze weather phenomena along the flight path, including turbulence, icing, and winds aloft, and advise on optimal cruising altitudes or diversions.",
    'regulatory_compliance_officer': "You are an AOCC regulatory compliance expert. Identify and ensure adherence to all relevant aviation regulations (ICAO, national), NOTAMs, TFRs, and airline policies.",
    'fuel_load_optimiser': "You are an AOCC fuel and load planning engineer. Calculate precise fuel requirements (including reserves) and provide critical guidance on aircraft weight and balance for safe and efficient operations.",
    'contingency_planning_specialist': "You are an AOCC contingency planning specialist. Develop comprehensive alternate airport strategies and emergency response plans for potential disruptions.",
    'passenger_coordinator': "You are an AOCC passenger services coordinator. Provide insights into passenger counts, special needs, and strategies for passenger communication and re-accommodation during irregular operations.",
    'crew_scheduler': "You are an AOCC crew scheduler. Provide crew availability, duty time limitations, rest requirements, and optimal crew pairings for flights.",
    'maintenance_coordinator': "You are an AOCC maintenance coordinator. Report on aircraft maintenance status, potential mechanical issues, required inspections, and ensure aircraft airworthiness and readiness.",
    'operational_synthesis_director': "You are the AOCC Operational Director. Your role is to synthesize all information from various specialized desks into a cohesive and actionable operational flight plan."
}


# --- Agent Service Class ---
class AgentService:
    def __init__(self, client, model_name, agent_roles):
        self.client = client
        self.model_name = model_name
        self.agent_roles = agent_roles

    def get_response(self, agent_name, user_prompt, conversation_history=None):
        """Gets an LLM response, acting as a specific agent, using the Mistral API."""
        system_message = self.agent_roles.get(agent_name, "You are a helpful AI assistant.")
        messages = [{"role": "system", "content": system_message}]
        if conversation_history:
            messages.extend(conversation_history)
        messages.append({"role": "user", "content": user_prompt})

        print(f"\n[Calling LLM as {agent_name} with prompt]: {user_prompt[:100]}...")
        try:
            chat_response = self.client.chat.complete(
                model=self.model_name,
                messages=messages,
            )
            return chat_response.choices[0].message.content
        except Exception as e:
            print(f"Error calling Mistral LLM for {agent_name}: {e}")
            return f"Error: {agent_name} could not provide a response due to an API error."

# --- Flight Class (representing a flight's state) ---
class Flight:
    def __init__(self, flight_id, departure_icao, destination_icao, aircraft_type):
        self.flight_id = flight_id
        self.departure_icao = departure_icao
        self.destination_icao = destination_icao
        self.aircraft_type = aircraft_type
        self.status = "Planned" # e.g., Planned, Active, Completed, Diverted, Delayed
        self.current_plan = {} # Stores the detailed plan
        self.plan_history = [] # To keep track of re-planning events
        self.last_update_time = datetime.now()
        # Add attributes for new agents
        self.passenger_info = "Not yet gathered"
        self.crew_info = "Not yet assigned"
        self.maintenance_status = "Not yet checked"


    def update_plan(self, new_plan_details, reason="Initial Planning"):
        """Updates the current flight plan and logs it."""
        self.current_plan = new_plan_details
        self.plan_history.append({"timestamp": datetime.now(), "reason": reason, "plan": new_plan_details})
        self.last_update_time = datetime.now()
        print(f"Flight {self.flight_id}: Plan updated - {reason}")

    def get_status_report(self):
        """Generates a summary of the flight's current status and plan."""
        report = f"--- Flight {self.flight_id} Status Report ---\n"
        report += f"  Route: {self.departure_icao} to {self.destination_icao} with {self.aircraft_type}\n"
        report += f"  Status: {self.status}\n"
        if self.current_plan:
            report += f"  Estimated Flight Time: {self.current_plan.get('flight_time_minutes', 'N/A')} minutes\n"
            report += f"  Estimated Fuel Required: {self.current_plan.get('fuel_required', 'N/A')} lbs\n"
            report += f"  Origin Weather: {self.current_plan.get('origin_weather', 'N/A')}\n"
            report += f"  Destination Weather: {self.current_plan.get('destination_weather', 'N/A')}\n"
            report += f"  Estimated Arrival: {self.current_plan.get('estimated_arrival_time', 'N/A')}\n"
        report += f"  Passengers: {self.passenger_info}\n"
        report += f"  Crew: {self.crew_info}\n"
        report += f"  Maintenance: {self.maintenance_status}\n"
        return report

# --- OCC Controller Class ---
class OCC_Controller:
    def __init__(self, mistral_client, airport_data, aircraft_data, agent_roles, model_name):
        self.tool_service = ToolService()
        self.agent_service = AgentService(mistral_client, model_name, agent_roles)
        self.airport_data = airport_data
        self.aircraft_data = aircraft_data
        self.flights = {} # Dictionary to store Flight objects, keyed by flight_id
        self.flight_id_counter = 0

    def create_flight(self, departure_icao, destination_icao, aircraft_type):
        """Creates a new flight entry and initiates planning."""
        self.flight_id_counter += 1
        flight_id = f"FL{self.flight_id_counter:04d}-{aircraft_type.replace(' ', '')}"
        new_flight = Flight(flight_id, departure_icao, destination_icao, aircraft_type)
        self.flights[flight_id] = new_flight
        print(f"\n[AOCC] Created new flight: {flight_id}")
        return flight_id

    def plan_flight_initial(self, flight_id, weather_severity='normal'):
        """
        Conducts initial comprehensive flight planning for a given flight ID.
        This is essentially the logic from the original `plan_flight` but
        updates the state of a `Flight` object.
        """
        flight = self.flights.get(flight_id)
        if not flight:
            print(f"Error: Flight {flight_id} not found.")
            return False

        dep_icao = flight.departure_icao
        dest_icao = flight.destination_icao
        ac_type = flight.aircraft_type

        print(f"\n--- Initiating Initial Planning for Flight {flight_id} ({dep_icao} to {dest_icao} with {ac_type}) ---")

        # Initial Data Gathering & Validation
        dep_airport_data = self.airport_data.get(dep_icao.upper())
        dest_airport_data = self.airport_data.get(dest_icao.upper())
        aircraft_data_obj = self.aircraft_data.get(ac_type.upper())

        if not dep_airport_data or not dest_airport_data or not aircraft_data_obj:
            print(f"Error: Invalid airport codes or aircraft type for flight {flight_id}.")
            flight.status = "Planning Failed"
            return False

        print("\n[AOCC Step 1: Initial Data Gathering & Validation]")
        print(f" Departure: {dep_airport_data['name']} ({dep_icao})")
        print(f" Destination: {dest_airport_data['name']} ({dest_icao})")
        print(f" Aircraft: {ac_type} (Speed: {aircraft_data_obj['speed']} mph, Fuel Rate: {aircraft_data_obj['fuel_consumption_rate']} lbs/hr)")

        flight_plan_details = {} # Temporary dict to collect agent responses

        # --- Step-by-step orchestration using conceptual agents ---

        # Agent: Flight Request Processor
        print("\n[AOCC Step 2: Flight Request Processor - Processing Request]")
        user_request_prompt = (
            f"The airline needs to plan a flight from {dep_icao} to {dest_icao} using a {ac_type}. "
            "A comprehensive operational plan considering all factors is required. "
            f"The initial assumption for destination weather is '{weather_severity}'. "
            "Summarize the core request and identify initial key operational data points needed for the AOCC."
        )
        response_content = self.agent_service.get_response('flight_request_processor', user_request_prompt)
        flight_plan_details['initial_request_processing'] = response_content

        # Agent: Aircraft Resources Agent
        print("\n[AOCC Step 3: Aircraft Resources Agent - Retrieving Data]")
        aircraft_prompt = f"Provide detailed performance and availability data for a {ac_type} from the airline's fleet, including its speed, fuel consumption rate, range, and optimal cruising altitude."
        response_content = self.agent_service.get_response('aircraft_resources_agent', aircraft_prompt)
        flight_plan_details['aircraft_performance'] = response_content

        # Agent: Airport Operations Agent
        print("\n[AOCC Step 4: Airport Operations Agent - Retrieving Data]")
        airport_prompt = f"Provide comprehensive operational details for {dep_icao} and {dest_icao} airports, including their full names, coordinates, available runways, and any current operational advisories or restrictions."
        response_content = self.agent_service.get_response('airport_ops_agent', airport_prompt)
        flight_plan_details['airport_info'] = response_content

        # Agent: Flight Dispatch Agent
        print("\n[AOCC Step 5: Flight Dispatch Agent - Calculating Route]")
        distance = self.tool_service.calculate_distance(dep_airport_data['lat'], dep_airport_data['lon'], dest_airport_data['lat'], dest_airport_data['lon'])
        flight_time_hours = distance / aircraft_data_obj['speed']
        route_prompt = (
            f"The direct flight distance between {dep_airport_data['name']} and {dest_airport_data['name']} is {distance:.2f} miles. "
            f"For a {ac_type} with a speed of {aircraft_data_obj['speed']} mph, the estimated flight time is {flight_time_hours:.2f} hours. "
            "Based on this, describe an optimal flight route, considering typical air corridors, and any initial operational considerations for dispatch."
        )
        response_content = self.agent_service.get_response('flight_dispatch_agent', route_prompt)
        flight_plan_details['route_calculation'] = response_content
        flight_plan_details['calculated_distance'] = distance
        flight_plan_details['estimated_flight_time_hours'] = flight_time_hours

        # Agent: Origin Weather Desk
        print("\n[AOCC Step 6: Origin Weather Desk - Getting Weather]")
        origin_weather_data = self.tool_service.get_simulated_weather(dep_icao, 'normal')
        origin_weather_prompt = f"Analyze the current and forecasted weather conditions at {dep_icao}: '{origin_weather_data}'. What are the operational implications for takeoff and initial climb phases?"
        response_content = self.agent_service.get_response('origin_weather_desk', origin_weather_prompt)
        flight_plan_details['origin_weather'] = response_content

        # Agent: Destination Weather Desk
        print("\n[AOCC Step 7: Destination Weather Desk - Getting Weather]")
        dest_weather_data = self.tool_service.get_simulated_weather(dest_icao, weather_severity)
        dest_weather_prompt = f"Analyze the current and forecasted weather conditions at {dest_icao}: '{dest_weather_data}'. What are the operational implications for approach, landing, and potential hazards?"
        response_content = self.agent_service.get_response('destination_weather_desk', dest_weather_prompt)
        flight_plan_details['destination_weather'] = response_content

        # Agent: En-Route Weather Desk
        print("\n[AOCC Step 8: En-Route Weather Desk - Analyzing Route Weather]")
        enroute_proxy_airport = random.choice(list(self.airport_data.keys()))
        enroute_weather_data = self.tool_service.get_simulated_weather(enroute_proxy_airport, random.choice(['normal', 'moderate', 'bad']))
        enroute_weather_prompt = f"Analyze potential en-route weather conditions for a flight from {dep_icao} to {dest_icao}, considering simulated conditions at {enroute_proxy_airport}: '{enroute_weather_data}'. Discuss implications for turbulence, icing, and winds aloft, and advise on optimal cruising altitudes or potential re-routing."
        response_content = self.agent_service.get_response('enroute_weather_desk', enroute_weather_prompt)
        flight_plan_details['enroute_weather'] = response_content

        # Agent: Regulatory Compliance Officer
        print("\n[AOCC Step 9: Regulatory Compliance Officer - Checking Regulations]")
        regulatory_prompt = f"For a flight from {dep_icao} to {dest_icao} with a {ac_type}, what key general aviation regulatory considerations (e.g., ICAO, national aviation authority regulations), current NOTAMs, and TFRs would apply to this airline operation?"
        response_content = self.agent_service.get_response('regulatory_compliance_officer', regulatory_prompt)
        flight_plan_details['regulatory_compliance'] = response_content

        # Agent: Fuel & Load Optimiser
        print("\n[AOCC Step 10: Fuel & Load Optimiser - Calculating Requirements]")
        fuel_required_calculated = flight_time_hours * aircraft_data_obj['fuel_consumption_rate']
        fuel_load_prompt = (
            f"Calculate the estimated fuel required for a {ac_type} flying {distance:.2f} miles, "
            f"with an estimated flight time of {flight_time_hours:.2f} hours and a fuel consumption rate of {aircraft_data_obj['fuel_consumption_rate']} lbs/hr. "
            f"The calculated fuel is {fuel_required_calculated:.2f} lbs. "
            "Provide critical guidance on aircraft weight and balance, including contingency fuel, for safe and efficient operations."
        )
        response_content = self.agent_service.get_response('fuel_load_optimiser', fuel_load_prompt)
        flight_plan_details['fuel_load'] = response_content
        flight_plan_details['estimated_fuel_required'] = fuel_required_calculated

        # Agent: Contingency Planning Specialist
        print("\n[AOCC Step 11: Contingency Planning Specialist - Developing Alternates]")
        alternate_airport_code = 'LHR' # Example: In a real system, this would be dynamically chosen
        alternate_weather_data = self.tool_service.get_simulated_weather(alternate_airport_code, 'normal')
        contingency_prompt = (
            f"Given the destination weather at {dest_icao} is '{weather_severity}' ({flight_plan_details.get('destination_weather', 'N/A')}), "
            f"and considering a {ac_type} (range: {aircraft_data_obj['range']} miles), "
            f"and a potential alternate airport {alternate_airport_code} with weather '{alternate_weather_data}', "
            "suggest suitable alternate airports and provide actionable advice on general contingency planning for this operational flight."
        )
        response_content = self.agent_service.get_response('contingency_planning_specialist', contingency_prompt)
        flight_plan_details['contingency_planning'] = response_content

        # Agent: Passenger Coordinator
        print("\n[AOCC Step 12: Passenger Coordinator - Assessing Passenger Information]")
        passenger_prompt = f"For flight {flight_id} ({dep_icao} to {dest_icao} with {ac_type}), provide insights into typical passenger counts for this aircraft type on an international route, any general special assistance considerations, and strategies for passenger communication during potential irregular operations (e.g., delays)."
        response_content = self.agent_service.get_response('passenger_coordinator', passenger_prompt)
        flight_plan_details['passenger_info'] = response_content
        flight.passenger_info = response_content # Update flight object

        # Agent: Crew Scheduler
        print("\n[AOCC Step 13: Crew Scheduler - Optimizing Crew Assignment]")
        crew_prompt = f"For flight {flight_id} ({dep_icao} to {dest_icao} with {ac_type}, estimated flight time {flight_time_hours:.2f} hours), provide guidance on optimal crew assignments, duty time limitations, and rest requirements, ensuring compliance and operational efficiency."
        response_content = self.agent_service.get_response('crew_scheduler', crew_prompt)
        flight_plan_details['crew_info'] = response_content
        flight.crew_info = response_content # Update flight object

        # Agent: Maintenance Coordinator
        print("\n[AOCC Step 14: Maintenance Coordinator - Confirming Aircraft Readiness]")
        maintenance_prompt = f"For aircraft {ac_type} assigned to flight {flight_id}, report on its current maintenance status, any recent significant maintenance events, and confirm its airworthiness and readiness for dispatch for an international flight."
        response_content = self.agent_service.get_response('maintenance_coordinator', maintenance_prompt)
        flight_plan_details['maintenance_status'] = response_content
        flight.maintenance_status = response_content # Update flight object


        # Final Synthesis by the Operational Synthesis Director
        print("\n[AOCC Step 15: Operational Synthesis Director - Final Synthesis of Operational Plan]")
        final_synthesis_prompt = (
            "As the AOCC Operational Director, synthesize the following information provided by various specialized AOCC desks into a comprehensive and actionable operational flight plan. "
            "Ensure it's well-structured with clear headings and covers all critical aspects for a flight from "
            f"{dep_icao} to {dest_icao} with a {ac_type}. "
            "Include estimated flight time and fuel, and a concise summary of all key AOCC findings and directives, including personnel and aircraft readiness.\n\n"
            f"**Flight Request Summary:** {flight_plan_details.get('initial_request_processing', 'N/A')}\n\n"
            f"**Aircraft Resources Assessment:** {flight_plan_details.get('aircraft_performance', 'N/A')}\n\n"
            f"**Airport Operational Data:** {flight_plan_details.get('airport_info', 'N/A')}\n\n"
            f"**Flight Dispatch Routing:** {flight_plan_details.get('route_calculation', 'N/A')}\n\n"
            f"**Origin Weather Brief:** {flight_plan_details.get('origin_weather', 'N/A')}\n\n"
            f"**Destination Weather Brief:** {flight_plan_details.get('destination_weather', 'N/A')}\n\n"
            f"**En-Route Weather Outlook:** {flight_plan_details.get('enroute_weather', 'N/A')}\n\n"
            f"**Regulatory Compliance Check:** {flight_plan_details.get('regulatory_compliance', 'N/A')}\n\n"
            f"**Fuel & Load Optimisation:** {flight_plan_details.get('fuel_load', 'N/A')}\n\n"
            f"**Contingency & Diversion Plan:** {flight_plan_details.get('contingency_planning', 'N/A')}\n\n"
            f"**Passenger Coordination Brief:** {flight_plan_details.get('passenger_info', 'N/A')}\n\n"
            f"**Crew Assignment & Compliance:** {flight_plan_details.get('crew_info', 'N/A')}\n\n"
            f"**Aircraft Maintenance Readiness:** {flight_plan_details.get('maintenance_status', 'N/A')}\n\n"
            f"**Calculated Direct Distance:** {distance:.2f} miles\n"
            f"**Estimated Flight Time:** {round(flight_time_hours * 60)} minutes\n"
            f"**Estimated Fuel Required:** {fuel_required_calculated:.2f} lbs\n"
            f"**Estimated Arrival Time:** {(datetime.now() + timedelta(minutes=round(flight_time_hours * 60))).strftime('%H:%M (local time)')}\n"
        )
        final_llm_plan_details = self.agent_service.get_response('operational_synthesis_director', final_synthesis_prompt)

        final_plan_summary = {
            'departure': dep_airport_data['name'],
            'destination': dest_airport_data['name'],
            'aircraft': ac_type,
            'distance': f"{distance:.2f}",
            'flight_time_minutes': round(flight_time_hours * 60),
            'fuel_required': f"{fuel_required_calculated:.2f}",
            'origin_weather': flight_plan_details.get('origin_weather', 'N/A'),
            'destination_weather': flight_plan_details.get('destination_weather', 'N/A'),
            'llm_plan_details': final_llm_plan_details,
            'estimated_arrival_time': (datetime.now() + timedelta(minutes=round(flight_time_hours * 60))).strftime('%H:%M (local time)')
        }
        flight.update_plan(final_plan_summary, reason="Initial Planning Completed")
        flight.status = "Planned"
        return True

    def monitor_flights(self):
        """Simulates ongoing monitoring of active flights by the AOCC."""
        print("\n[AOCC] Monitoring active flights...")
        for flight_id, flight in self.flights.items():
            if flight.status == "Planned" or flight.status == "Active":
                print(f"  Monitoring {flight.flight_id} - Status: {flight.status}")
                # Example: Check for stale plan (e.g., plan older than 24 hours)
                if (datetime.now() - flight.last_update_time).total_seconds() > 3600 * 24:
                    print(f"    Flight {flight_id}: Plan is stale, considering re-planning.")
                    # In a real AOCC, this might trigger an alert to an operator
                    # or automatically initiate a minor replan.
                    # self.replan_flight(flight_id, reason="Stale plan due to time lapse")

    def replan_flight(self, flight_id, new_weather_severity=None, reason="Manual Re-planning Triggered"):
        """
        Triggers a re-planning cycle for a specific flight,
        e.g., due to updated weather, NOTAMs, or operational changes.
        """
        flight = self.flights.get(flight_id)
        if not flight:
            print(f"Error: Flight {flight_id} not found for re-planning.")
            return False

        print(f"\n--- Initiating Re-planning for Flight {flight_id} ({reason}) ---")
        current_weather_severity = new_weather_severity if new_weather_severity else 'normal'

        success = self.plan_flight_initial(flight_id, weather_severity=current_weather_severity)
        if success:
            flight.status = "Planned (Re-planned)"
            print(f"Re-planning for Flight {flight_id} completed successfully.")
        else:
            flight.status = "Re-planning Failed"
            print(f"Re-planning for Flight {flight_id} failed.")
        return success

    def get_flight_report(self, flight_id):
        """Retrieves a comprehensive report for a specific flight."""
        flight = self.flights.get(flight_id)
        if flight:
            print(flight.get_status_report())
            if flight.current_plan and flight.current_plan.get('llm_plan_details'):
                print("--- Full LLM Synthesized Operational Plan Details ---")
                print(flight.current_plan['llm_plan_details'])
        else:
            print(f"Flight {flight_id} not found.")

# --- Main Execution Block (simulating AOCC operations) ---
if __name__ == "__main__":
    # Ensure Mistral API client is initialized
    try:
        import colab_env # This line might cause issues if not in Colab
        api_key = os.environ.get("MISTRAL_API_KEY")
        if not api_key:
            raise ValueError("MISTRAL_API_KEY environment variable not set.")
        client = Mistral(api_key=api_key)
    except (ImportError, ValueError) as e:
        print(f"Error initializing Mistral client: {e}. Please ensure `colab-env` is installed and MISTRAL_API_KEY is set.")
        exit()

    # Initialize the AOCC Controller with the new agent roles
    aocc_center = OCC_Controller(client, AIRPORT_DATA, AIRCRAFT_DATA, AOCC_AGENT_ROLES, MISTRAL_MODEL)

    # Scenario 1: Initial Flight Planning for a new flight request
    print("\n--- AOCC Scenario 1: Processing New Flight Request for Operational Planning ---")
    flight1_id = aocc_center.create_flight("YUL", "PVG", "BOEING 777")
    aocc_center.plan_flight_initial(flight1_id)
    aocc_center.get_flight_report(flight1_id)

    # Scenario 2: Simulate a Weather-driven Re-planning (Operational Adjustment)
    print("\n\n--- AOCC Scenario 2: Operational Adjustment due to Weather Change ---")
    new_dest_severity = random.choice(['moderate', 'bad'])
    print(f"[AOCC] Detecting a change to '{new_dest_severity}' weather at destination for Flight {flight1_id}. Initiating operational re-planning.")
    aocc_center.replan_flight(flight1_id, new_weather_severity=new_dest_severity, reason=f"Dynamic weather update to '{new_dest_severity}' at PVG")
    aocc_center.get_flight_report(flight1_id)

    # Scenario 3: Add another flight to the operational schedule
    print("\n\n--- AOCC Scenario 3: Adding another Flight to Operational Schedule ---")
    flight2_id = aocc_center.create_flight("JFK", "LHR", "AIRBUS A320")
    aocc_center.plan_flight_initial(flight2_id)
    aocc_center.get_flight_report(flight2_id)

    # Scenario 4: AOCC Global Monitoring Check
    print("\n\n--- AOCC Scenario 4: Global Flight Monitoring Check ---")
    aocc_center.monitor_flights()

    # Further AOCC scenarios could involve:
    # - Real-time flight tracking updates (position, speed, altitude) and deviation detection
    # - Crew scheduling and duty time management integrations (more detailed)
    # - Gate assignment and ground handling coordination
    # - Communication with air traffic control (simulated)
    # - Passenger re-accommodation planning during disruptions (more detailed)


--- AOCC Scenario 1: Processing New Flight Request for Operational Planning ---

[AOCC] Created new flight: FL0001-BOEING777

--- Initiating Initial Planning for Flight FL0001-BOEING777 (YUL to PVG with BOEING 777) ---

[AOCC Step 1: Initial Data Gathering & Validation]
 Departure: Montréal-Trudeau International Airport (YUL)
 Destination: Shanghai Pudong International Airport (PVG)
 Aircraft: BOEING 777 (Speed: 550 mph, Fuel Rate: 3000 lbs/hr)

[AOCC Step 2: Flight Request Processor - Processing Request]

[Calling LLM as flight_request_processor with prompt]: The airline needs to plan a flight from YUL to PVG using a BOEING 777. A comprehensive operational p...

[AOCC Step 3: Aircraft Resources Agent - Retrieving Data]

[Calling LLM as aircraft_resources_agent with prompt]: Provide detailed performance and availability data for a BOEING 777 from the airline's fleet, includ...

[AOCC Step 4: Airport Operations Agent - Retrieving Data]

[Calling LLM as airport_ops_agent with prompt]: P